In [48]:
import pandas as pd
from scipy.stats import chi2_contingency

In [49]:
df_survey_counts = pd.DataFrame({
    "Green": [127],
    "Invalid vote": [3],
    "Red": [70]
}, index=["Votes"])

df_survey_shares = (df_survey_counts / df_survey_counts.sum(axis=1).values[0] * 100).round(1)
df_survey_shares.index = ["Share (%)"]

df_survey_combined = pd.concat([df_survey_counts, df_survey_shares])

df_survey_combined


,Green,Invalid vote,Red
Votes,127.0,3.0,70.0
Share (%),63.5,1.5,35.0


In [50]:
df_results_counts = pd.DataFrame({
    "Green": [612],
    "Invalid vote": [19],
    "Red": [408]
}, index=["Votes"])

df_results_shares = (df_results_counts / df_results_counts.sum(axis=1).values[0] * 100).round(1)
df_results_shares.index = ["Share (%)"]

df_results_combined = pd.concat([df_results_counts, df_results_shares])

# convert only votes as int
df_results_combined

,Green,Invalid vote,Red
Votes,612.0,19.0,408.0
Share (%),58.9,1.8,39.3


In [51]:
df_change = (df_results_shares - df_survey_shares).round(1)
df_change.index = ["Change (pp)"]
df_change = df_change.applymap(lambda x: f"+{x}" if x > 0 else f"{x}")
df_change

/var/folders/q8/75fwdtys4yb1j6k3ghx7dn340000gn/T/ipykernel_17657/3277468289.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_change = df_change.applymap(lambda x: f"+{x}" if x > 0 else f"{x}")


,Green,Invalid vote,Red
Change (pp),-4.6,+0.3,+4.3


In [52]:
# Observed counts
observed = pd.DataFrame({
    "Green": [127, 612],
    "Invalid vote": [3, 19],
    "Red": [70, 408]
}, index=["Survey", "Results"])

# Chi-squared test
chi2, p, dof, expected = chi2_contingency(observed)

print("Chi-squared:", round(chi2, 3))
print("Degrees of freedom:", dof)
print("p-value:", round(p, 4))
print("\nExpected counts:\n", pd.DataFrame(expected, index=observed.index, columns=observed.columns))

Chi-squared: 1.488
Degrees of freedom: 2
p-value: 0.4751

Expected counts:
              Green  Invalid vote         Red
Survey   119.28975      3.551251   77.158999
Results  619.71025     18.448749  400.841001
